In [2]:
# Copyright 2021 The TensorFlow Authors.

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# modified by Jeonghun Kim


#1 Preparation
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import glob
import sys
import tempfile
import tqdm

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



#2 Load MoveNet Thunder model
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples\\lite\\examples\\pose_estimation\\raspberry_pi')
sys.path.append(pose_sample_rpi_path)
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.
 
  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.
 
  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape
 
  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)
 
  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)

  return person



#3 Functions to visualize the pose estimation results.
def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.
 
  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.
 
  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])
  
  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)
 
  if close_figure:
    plt.close(fig)
 
  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np



#4 Code to load the images, detect pose landmarks and save them into a CSV file

class MoveNetPreprocessor(object):
  """Helper class to preprocess pose sample images for classification."""
 
  def __init__(self,
               images_in_folder,
               images_out_folder,
               csvs_out_path):
    """Creates a preprocessor to detection pose from images and save as CSV.

    Args:
      images_in_folder: Path to the folder with the input images. It should
        follow this structure:
        yoga_poses
        |__ downdog
            |______ 00000128.jpg
            |______ 00000181.bmp
            |______ ...
        |__ goddess
            |______ 00000243.jpg
            |______ 00000306.jpg
            |______ ...
        ...
      images_out_folder: Path to write the images overlay with detected
        landmarks. These images are useful when you need to debug accuracy
        issues.
      csvs_out_path: Path to write the CSV containing the detected landmark
        coordinates and label of each image that can be used to train a pose
        classification model.
    """
    self._images_in_folder = images_in_folder
    self._images_out_folder = images_out_folder
    self._csvs_out_path = csvs_out_path
    self._messages = []

    # Create a temp dir to store the pose CSVs per class
    self._csvs_out_folder_per_class = tempfile.mkdtemp()
 
    # Get list of pose classes and print image statistics
    self._pose_class_names = sorted(
        [n for n in os.listdir(self._images_in_folder) if not n.startswith('.')]
        )
    
  def process(self, per_pose_class_limit=None, detection_threshold=0.1):
    """Preprocesses images in the given folder.
    Args:
      per_pose_class_limit: Number of images to load. As preprocessing usually
        takes time, this parameter can be specified to make the reduce of the
        dataset for testing.
      detection_threshold: Only keep images with all landmark confidence score
        above this threshold.
    """
    # Loop through the classes and preprocess its images
    for pose_class_name in self._pose_class_names:
      print('Preprocessing', pose_class_name, file=sys.stderr)

      # Paths for the pose class.
      images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
      images_out_folder = os.path.join(self._images_out_folder, pose_class_name)
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  pose_class_name + '.csv')
      if not os.path.exists(images_out_folder):
        os.makedirs(images_out_folder)
 
      # Detect landmarks in each image and write it to a CSV file
      with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, 
                                    delimiter=',', 
                                    quoting=csv.QUOTE_MINIMAL)
        # Get list of images
        image_names = sorted(
            [n for n in os.listdir(images_in_folder) if not n.startswith('.')])
        if per_pose_class_limit is not None:
          image_names = image_names[:per_pose_class_limit]

        valid_image_count = 0
 
        # Detect pose landmarks from each image
        for image_name in tqdm.tqdm(image_names):
          image_path = os.path.join(images_in_folder, image_name)

          try:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
          except:
            self._messages.append('Skipped ' + image_path + '. Invalid image.')
            continue
          else:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
            image_height, image_width, channel = image.shape
          
          # Skip images that isn't RGB because Movenet requires RGB images
          if channel != 3:
            self._messages.append('Skipped ' + image_path +
                                  '. Image isn\'t in RGB format.')
            continue
          person = detect(image)
          
          # Save landmarks if all landmarks were detected
          min_landmark_score = min(
              [keypoint.score for keypoint in person.keypoints])
          should_keep_image = min_landmark_score >= detection_threshold
          if not should_keep_image:
            self._messages.append('Skipped ' + image_path +
                                  '. No pose was confidentlly detected.')
            continue

          valid_image_count += 1


          # Draw the prediction result on top of the image for debugging later
          """
          output_overlay = draw_prediction_on_image(
              image.numpy().astype(np.uint8), person, 
              close_figure=True, keep_input_size=True)
        
          # Write detection result into an image file
          output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
          cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
          """
        
          # Get landmarks and scale it to the same size as the input image
          pose_landmarks = np.array(
              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints],
              dtype=np.float32)

          # Write the landmark coordinates to its per-class CSV file
          coordinates = pose_landmarks.flatten().astype(np.str).tolist()
          csv_out_writer.writerow([image_name] + coordinates)

        if not valid_image_count:
          raise RuntimeError(
              'No valid images found for the "{}" class.'
              .format(pose_class_name))
      
    # Print the error message collected during preprocessing.
    print('\n'.join(self._messages))

    # Combine all per-class CSVs into a single output file
    all_landmarks_df = self._all_landmarks_as_dataframe()
    all_landmarks_df.to_csv(self._csvs_out_path, index=False)

  def class_names(self):
    """List of classes found in the training dataset."""
    return self._pose_class_names
  
  def _all_landmarks_as_dataframe(self):
    """Merge all per-class CSVs into a single dataframe."""
    total_df = None
    for class_index, class_name in enumerate(self._pose_class_names):
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  class_name + '.csv')
      per_class_df = pd.read_csv(csv_out_path, header=None)
      
      # Add the labels
      per_class_df['class_no'] = [class_index]*len(per_class_df)
      per_class_df['class_name'] = [class_name]*len(per_class_df)

      # Append the folder name to the filename column (first column)
      per_class_df[per_class_df.columns[0]] = (os.path.join(class_name, '') 
        + per_class_df[per_class_df.columns[0]].astype(str))

      if total_df is None:
        # For the first class, assign its data to the total dataframe
        total_df = per_class_df
      else:
        # Concatenate each class's data into the total dataframe
        total_df = pd.concat([total_df, per_class_df], axis=0)
 
    list_name = [[bodypart.name + '_x', bodypart.name + '_y', 
                  bodypart.name + '_score'] for bodypart in BodyPart] 
    header_name = []
    for columns_name in list_name:
      header_name += columns_name
    header_name = ['file_name'] + header_name
    header_map = {total_df.columns[i]: header_name[i] 
                  for i in range(len(header_name))}
 
    total_df.rename(header_map, axis=1, inplace=True)

    return total_df

In [22]:
#5.1 dataset processing-train

IMAGES_ROOT = 'split'

classname = {'jumpingjacks', 'pullup', 'pushup', 'situp', 'squat'}
count = 0

for c in classname:
  images_in_train_folder = os.path.join(IMAGES_ROOT, 'train/'+c)
  images_out_train_folder = 'poses_images_out_train'
  csvs_out_train_path = 'train_data.csv'

  preprocessor = MoveNetPreprocessor(
      images_in_folder=images_in_train_folder,
      images_out_folder=images_out_train_folder,
      csvs_out_path=csvs_out_train_path,
  )

  preprocessor.process(per_pose_class_limit=None)
  f = open('train_data.csv','r')
  rdr = csv.reader(f)
  lines = []
  for line in rdr:
        if line[52] != "class_no":
            line[52] = str(count)
        if line[53] != "class_name":
            line[53] = c
        lines.append(line)
  
  f = open('train_data_'+c+'.csv','w',newline='') 
  wr = csv.writer(f)
  wr.writerows(lines)
  print('train_data_'+c+'.csv is saved, classno:'+str(count))
    
  f.close()
  count+=1

Preprocessing 1350
  0%|          | 0/43 [00:00<?, ?it/s]C:\Users\jeokj\AppData\Local\Temp\ipykernel_9820\1933474597.py:248: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coordinates = pose_landmarks.flatten().astype(np.str).tolist()
100%|██████████| 43/43 [00:02<00:00, 16.66it/s]
Preprocessing 1351
100%|██████████| 44/44 [00:02<00:00, 16.84it/s]
Preprocessing 1352
100%|██████████| 46/46 [00:02<00:00, 16.94it/s]
Preprocessing 1353
100%|██████████| 41/41 [00:02<00:00, 16.94it/s]
Preprocessing 1354
100%|██████████| 30/30 [00:01<00:00, 16.64it/s]
Preprocessing 1355
100%|██████████| 29/29 [00:01<00:00, 16.94it/s]
Preprocessing 1356
100%|██████████| 50/50 [00:02<00:00, 16.74i

Skipped split\train/pushup\1352\000009.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1352\000038.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1352\000041.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1354\000013.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1354\000026.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000001.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000005.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000007.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000008.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000009.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000010.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000012.jpg. No pose was confidentlly detected.
Skipped split\train/pushup\1355\000013.jpg. No pose 

Preprocessing 1560


train_data_pushup.csv is saved, classno:0


  0%|          | 0/79 [00:00<?, ?it/s]C:\Users\jeokj\AppData\Local\Temp\ipykernel_9820\1933474597.py:248: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coordinates = pose_landmarks.flatten().astype(np.str).tolist()
100%|██████████| 79/79 [00:04<00:00, 16.61it/s]
Preprocessing 1561
100%|██████████| 70/70 [00:04<00:00, 16.60it/s]
Preprocessing 1562
100%|██████████| 64/64 [00:03<00:00, 16.63it/s]
Preprocessing 1563
100%|██████████| 65/65 [00:03<00:00, 16.57it/s]
Preprocessing 1564
100%|██████████| 152/152 [00:09<00:00, 16.71it/s]
Preprocessing 1565
100%|██████████| 95/95 [00:05<00:00, 16.60it/s]
Preprocessing 1566
100%|██████████| 194/194 [00:11<00:00, 16.74it/s]
Preprocess

...

Skipped split\train/squat\1663\000006.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1663\000069.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1663\000084.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1663\000085.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1663\000086.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1663\000087.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000001.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000002.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000005.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000006.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000008.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000009.jpg. No pose was confidentlly detected.
Skipped split\train/squat\1664\000026.jpg. No pose was confident

Preprocessing 1150


train_data_squat.csv is saved, classno:3


 20%|██        | 8/40 [00:00<00:01, 16.68it/s]C:\Users\jeokj\AppData\Local\Temp\ipykernel_9820\1933474597.py:248: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coordinates = pose_landmarks.flatten().astype(np.str).tolist()
100%|██████████| 40/40 [00:02<00:00, 16.59it/s]
Preprocessing 1151
100%|██████████| 38/38 [00:02<00:00, 16.66it/s]
Preprocessing 1152
100%|██████████| 44/44 [00:02<00:00, 16.56it/s]
Preprocessing 1153
100%|██████████| 38/38 [00:02<00:00, 16.67it/s]
Preprocessing 1154
100%|██████████| 72/72 [00:04<00:00, 16.57it/s]
Preprocessing 1155
100%|██████████| 62/62 [00:03<00:00, 16.71it/s]
Preprocessing 1156
100%|██████████| 79/79 [00:04<00:00, 16.74it/s]
Prepro

RuntimeError: No valid images found for the "1206" class.

In [23]:
#5.2 dataset processing-test

IMAGES_ROOT = 'split'

count = 0

for c in classname:
  images_in_test_folder = os.path.join(IMAGES_ROOT, 'test/'+c)
  images_out_test_folder = 'poses_images_out_test'
  csvs_out_test_path = 'test_data.csv'

  preprocessor = MoveNetPreprocessor(
      images_in_folder=images_in_test_folder,
      images_out_folder=images_out_test_folder,
      csvs_out_path=csvs_out_test_path,
  )

  preprocessor.process(per_pose_class_limit=None)
  f = open('test_data.csv','r')
  rdr = csv.reader(f)
  lines = []
  for line in rdr:
        if line[52] != "class_no":
            line[52] = str(count)
        if line[53] != "class_name":
            line[53] = c
        lines.append(line)
  
  f = open('test_data_'+c+'.csv','w',newline='') 
  wr = csv.writer(f)
  wr.writerows(lines)
  print('test_data_'+c+'.csv is saved, classno:'+str(count))
    
  f.close()
  count+=1

Preprocessing 1511
  2%|▏         | 2/83 [00:00<00:04, 16.53it/s]C:\Users\jeokj\AppData\Local\Temp\ipykernel_9820\1933474597.py:248: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coordinates = pose_landmarks.flatten().astype(np.str).tolist()
100%|██████████| 83/83 [00:05<00:00, 15.92it/s]
Preprocessing 1512
100%|██████████| 98/98 [00:06<00:00, 16.03it/s]
Preprocessing 1513
100%|██████████| 29/29 [00:01<00:00, 16.33it/s]
Preprocessing 1514
100%|██████████| 90/90 [00:05<00:00, 16.11it/s]
Preprocessing 1515
100%|██████████| 79/79 [00:04<00:00, 16.06it/s]
Preprocessing 1517
100%|██████████| 40/40 [00:02<00:00, 15.84it/s]
Preprocessing 1518
100%|██████████| 52/52 [00:03<00:00

Skipped split\test/pushup\1511\000001.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000002.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000003.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000010.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000012.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000013.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000014.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000016.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000017.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000018.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000024.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000025.jpg. No pose was confidentlly detected.
Skipped split\test/pushup\1511\000026.jpg. No pose was confident

Preprocessing 1639


test_data_pushup.csv is saved, classno:0


  0%|          | 0/67 [00:00<?, ?it/s]C:\Users\jeokj\AppData\Local\Temp\ipykernel_9820\1933474597.py:248: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coordinates = pose_landmarks.flatten().astype(np.str).tolist()
100%|██████████| 67/67 [00:04<00:00, 14.77it/s]
Preprocessing 1640
100%|██████████| 59/59 [00:03<00:00, 15.56it/s]
Preprocessing 1641
100%|██████████| 53/53 [00:03<00:00, 15.43it/s]
Preprocessing 1642
100%|██████████| 44/44 [00:02<00:00, 15.49it/s]
Preprocessing 1643
100%|██████████| 63/63 [00:04<00:00, 15.39it/s]
Preprocessing 1644
100%|██████████| 62/62 [00:03<00:00, 15.73it/s]
Preprocessing 1645
100%|██████████| 242/242 [00:15<00:00, 16.10it/s]
Preprocessin

...

Skipped split\test/squat\1846\000059.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1846\000064.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1846\000065.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1846\000081.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000004.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000024.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000028.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000031.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000036.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000037.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000043.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000045.jpg. No pose was confidentlly detected.
Skipped split\test/squat\1847\000046.jpg. No pose was confidentlly detected.

Preprocessing 1298


test_data_squat.csv is saved, classno:3


  0%|          | 0/55 [00:00<?, ?it/s]C:\Users\jeokj\AppData\Local\Temp\ipykernel_9820\1933474597.py:248: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  coordinates = pose_landmarks.flatten().astype(np.str).tolist()
100%|██████████| 55/55 [00:03<00:00, 15.68it/s]
Preprocessing 1299
100%|██████████| 44/44 [00:02<00:00, 15.97it/s]
Preprocessing 1300
100%|██████████| 40/40 [00:02<00:00, 15.66it/s]
Preprocessing 1301
100%|██████████| 91/91 [00:05<00:00, 15.96it/s]
Preprocessing 1302
100%|██████████| 80/80 [00:05<00:00, 15.84it/s]
Preprocessing 1303
100%|██████████| 144/144 [00:08<00:00, 16.10it/s]
Preprocessing 1304
100%|██████████| 34/34 [00:02<00:00, 16.22it/s]
Preprocessin

Skipped split\test/pullup\1299\000021.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1299\000022.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1299\000023.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1299\000025.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1301\000088.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000001.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000002.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000003.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000004.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000005.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000006.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000007.jpg. No pose was confidentlly detected.
Skipped split\test/pullup\1302\000008.jpg. No pose was confident

In [5]:
#5.3 dataset processing-csv process



# merging the files
files_joined = os.path.join("train_data_*.csv")

# Return a list of all joined files
list_files = glob.glob(files_joined)

print("** Merging multiple csv files into a single pandas dataframe **")
# Merge files by joining all files
dataframe = pd.concat(map(pd.read_csv, list_files))
print(dataframe)

dataframe.to_csv('./train_data.csv', index=False)



# merging the files
files_joined = os.path.join("test_data_*.csv")

# Return a list of all joined files
list_files = glob.glob(files_joined)

print("** Merging multiple csv files into a single pandas dataframe **")
# Merge files by joining all files
dataframe = pd.concat(map(pd.read_csv, list_files))
print(dataframe)

dataframe.to_csv('./test_data.csv', index=False)

** Merging multiple csv files into a single pandas dataframe **
            file_name  NOSE_x  NOSE_y  NOSE_score  LEFT_EYE_x  LEFT_EYE_y  \
0     1350\000001.jpg   121.0   110.0    0.676941       123.0        96.0   
1     1350\000002.jpg   119.0   110.0    0.685184       123.0        96.0   
2     1350\000003.jpg   117.0   112.0    0.552196       122.0        98.0   
3     1350\000004.jpg   117.0   106.0    0.557192       118.0        95.0   
4     1350\000005.jpg   123.0   104.0    0.388228       123.0        93.0   
...               ...     ...     ...         ...         ...         ...   
7937  1297\000079.jpg   227.0   117.0    0.745216       232.0       114.0   
7938  1297\000080.jpg   227.0   120.0    0.572531       231.0       116.0   
7939  1297\000081.jpg   227.0   122.0    0.508951       231.0       118.0   
7940  1297\000082.jpg   226.0   125.0    0.616244       231.0       121.0   
7941  1297\000083.jpg   226.0   125.0    0.670333       231.0       121.0   

      LEFT_